# Lab 35: Lateral Movement Detection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab35_lateral_movement.ipynb)

Detect adversary movement through enterprise networks.

## Learning Objectives
- Authentication anomaly detection
- Remote execution identification (PsExec, WMI, WinRM)
- Graph-based attack path analysis
- Windows security event correlation

**Next:** Lab 36 (Threat Actor Profiling)

In [ ]:
!pip install pandas networkx matplotlib anthropic -q

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime, timedelta
from typing import List, Dict

## 1. Authentication Event Analysis

In [ ]:
# Comprehensive Windows authentication and lateral movement dataset
# Simulates a realistic enterprise environment under attack

from dataclasses import dataclass
from typing import Optional
import random

# Enterprise network topology
ENTERPRISE_HOSTS = {
    # Domain Controllers
    "DC01": {
        "type": "domain_controller",
        "os": "Windows Server 2019",
        "criticality": "critical",
        "subnet": "10.1.1.0/24",
    },
    "DC02": {
        "type": "domain_controller",
        "os": "Windows Server 2019",
        "criticality": "critical",
        "subnet": "10.1.1.0/24",
    },
    # Servers
    "FS01": {
        "type": "file_server",
        "os": "Windows Server 2019",
        "criticality": "high",
        "subnet": "10.1.2.0/24",
    },
    "FS02": {
        "type": "file_server",
        "os": "Windows Server 2016",
        "criticality": "high",
        "subnet": "10.1.2.0/24",
    },
    "DB01": {
        "type": "database",
        "os": "Windows Server 2019",
        "criticality": "critical",
        "subnet": "10.1.3.0/24",
    },
    "DB02": {
        "type": "database",
        "os": "Windows Server 2019",
        "criticality": "critical",
        "subnet": "10.1.3.0/24",
    },
    "WEB01": {
        "type": "web_server",
        "os": "Windows Server 2019",
        "criticality": "high",
        "subnet": "10.1.4.0/24",
    },
    "WEB02": {
        "type": "web_server",
        "os": "Windows Server 2019",
        "criticality": "high",
        "subnet": "10.1.4.0/24",
    },
    "EXCH01": {
        "type": "exchange",
        "os": "Windows Server 2019",
        "criticality": "critical",
        "subnet": "10.1.5.0/24",
    },
    "BACKUP01": {
        "type": "backup",
        "os": "Windows Server 2019",
        "criticality": "high",
        "subnet": "10.1.6.0/24",
    },
    "SCCM01": {
        "type": "sccm",
        "os": "Windows Server 2019",
        "criticality": "critical",
        "subnet": "10.1.7.0/24",
    },
    "JUMP01": {
        "type": "jump_server",
        "os": "Windows Server 2019",
        "criticality": "high",
        "subnet": "10.1.8.0/24",
    },
    # Workstations (by department)
    "WS-IT-001": {
        "type": "workstation",
        "os": "Windows 11",
        "criticality": "medium",
        "subnet": "10.2.1.0/24",
        "dept": "IT",
    },
    "WS-IT-002": {
        "type": "workstation",
        "os": "Windows 11",
        "criticality": "medium",
        "subnet": "10.2.1.0/24",
        "dept": "IT",
    },
    "WS-FIN-001": {
        "type": "workstation",
        "os": "Windows 10",
        "criticality": "high",
        "subnet": "10.2.2.0/24",
        "dept": "Finance",
    },
    "WS-FIN-002": {
        "type": "workstation",
        "os": "Windows 10",
        "criticality": "high",
        "subnet": "10.2.2.0/24",
        "dept": "Finance",
    },
    "WS-HR-001": {
        "type": "workstation",
        "os": "Windows 10",
        "criticality": "high",
        "subnet": "10.2.3.0/24",
        "dept": "HR",
    },
    "WS-ENG-001": {
        "type": "workstation",
        "os": "Windows 11",
        "criticality": "medium",
        "subnet": "10.2.4.0/24",
        "dept": "Engineering",
    },
    "WS-ENG-002": {
        "type": "workstation",
        "os": "Windows 11",
        "criticality": "medium",
        "subnet": "10.2.4.0/24",
        "dept": "Engineering",
    },
    "WS-EXEC-001": {
        "type": "workstation",
        "os": "Windows 11",
        "criticality": "critical",
        "subnet": "10.2.5.0/24",
        "dept": "Executive",
    },
}

# User accounts
USERS = {
    # Regular users
    "jsmith": {"type": "user", "department": "Engineering", "admin": False},
    "mjones": {"type": "user", "department": "Finance", "admin": False},
    "alee": {"type": "user", "department": "HR", "admin": False},
    "bwilson": {"type": "user", "department": "IT", "admin": False},
    "cwhite": {"type": "user", "department": "Executive", "admin": False},
    # IT Admins
    "it_admin1": {"type": "admin", "department": "IT", "admin": True, "scope": "tier1"},
    "it_admin2": {"type": "admin", "department": "IT", "admin": True, "scope": "tier2"},
    # Domain Admins
    "da_john": {"type": "domain_admin", "department": "IT", "admin": True, "scope": "domain"},
    "da_mary": {"type": "domain_admin", "department": "IT", "admin": True, "scope": "domain"},
    # Service Accounts
    "svc_backup": {"type": "service", "department": "IT", "admin": False, "purpose": "backup"},
    "svc_sql": {"type": "service", "department": "IT", "admin": False, "purpose": "database"},
    "svc_web": {"type": "service", "department": "IT", "admin": False, "purpose": "web"},
    "svc_sccm": {"type": "service", "department": "IT", "admin": True, "purpose": "sccm"},
}

# Comprehensive authentication events including attack patterns
AUTH_EVENTS = [
    # === NORMAL BASELINE ACTIVITY ===
    # Regular user logins
    {
        "event_id": 4624,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "dst_host": "FS01",
        "logon_type": 3,
        "time": "2024-01-15 08:30:00",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "Kerberos",
    },
    {
        "event_id": 4624,
        "user": "mjones",
        "src_host": "WS-FIN-001",
        "dst_host": "FS01",
        "logon_type": 3,
        "time": "2024-01-15 08:35:00",
        "src_ip": "10.2.2.10",
        "status": "success",
        "auth_package": "Kerberos",
    },
    {
        "event_id": 4624,
        "user": "alee",
        "src_host": "WS-HR-001",
        "dst_host": "FS02",
        "logon_type": 3,
        "time": "2024-01-15 08:40:00",
        "src_ip": "10.2.3.10",
        "status": "success",
        "auth_package": "Kerberos",
    },
    # IT admin normal activity
    {
        "event_id": 4624,
        "user": "it_admin1",
        "src_host": "JUMP01",
        "dst_host": "DC01",
        "logon_type": 3,
        "time": "2024-01-15 09:00:00",
        "src_ip": "10.1.8.10",
        "status": "success",
        "auth_package": "Kerberos",
    },
    {
        "event_id": 4624,
        "user": "it_admin1",
        "src_host": "JUMP01",
        "dst_host": "WEB01",
        "logon_type": 3,
        "time": "2024-01-15 09:15:00",
        "src_ip": "10.1.8.10",
        "status": "success",
        "auth_package": "Kerberos",
    },
    # Service account normal activity
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "BACKUP01",
        "dst_host": "FS01",
        "logon_type": 3,
        "time": "2024-01-15 02:00:00",
        "src_ip": "10.1.6.10",
        "status": "success",
        "auth_package": "NTLM",
    },
    {
        "event_id": 4624,
        "user": "svc_sql",
        "src_host": "WEB01",
        "dst_host": "DB01",
        "logon_type": 3,
        "time": "2024-01-15 08:00:00",
        "src_ip": "10.1.4.10",
        "status": "success",
        "auth_package": "Kerberos",
    },
    # === ATTACK CHAIN - Initial Compromise ===
    # Attacker gains access via phishing to jsmith's workstation
    {
        "event_id": 4624,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "dst_host": "WS-ENG-001",
        "logon_type": 2,
        "time": "2024-01-15 10:00:00",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "Negotiate",
        "attack_phase": "initial_access",
    },
    # === RECONNAISSANCE ===
    # Unusual DC queries from workstation
    {
        "event_id": 4624,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "dst_host": "DC01",
        "logon_type": 3,
        "time": "2024-01-15 10:05:00",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "Kerberos",
        "attack_phase": "reconnaissance",
    },
    {
        "event_id": 4624,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "dst_host": "DC02",
        "logon_type": 3,
        "time": "2024-01-15 10:05:30",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "Kerberos",
        "attack_phase": "reconnaissance",
    },
    # === CREDENTIAL ACCESS - Kerberoasting ===
    # Multiple TGS requests for service accounts (4769 events)
    {
        "event_id": 4769,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "service": "svc_sql/DB01",
        "time": "2024-01-15 10:10:00",
        "src_ip": "10.2.4.10",
        "encryption": "0x17",
        "ticket_options": "0x40810000",
        "attack_phase": "kerberoasting",
    },
    {
        "event_id": 4769,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "service": "svc_backup/BACKUP01",
        "time": "2024-01-15 10:10:05",
        "src_ip": "10.2.4.10",
        "encryption": "0x17",
        "ticket_options": "0x40810000",
        "attack_phase": "kerberoasting",
    },
    {
        "event_id": 4769,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "service": "svc_web/WEB01",
        "time": "2024-01-15 10:10:10",
        "src_ip": "10.2.4.10",
        "encryption": "0x17",
        "ticket_options": "0x40810000",
        "attack_phase": "kerberoasting",
    },
    {
        "event_id": 4769,
        "user": "jsmith",
        "src_host": "WS-ENG-001",
        "service": "svc_sccm/SCCM01",
        "time": "2024-01-15 10:10:15",
        "src_ip": "10.2.4.10",
        "encryption": "0x17",
        "ticket_options": "0x40810000",
        "attack_phase": "kerberoasting",
    },
    # === PRIVILEGE ESCALATION - Cracked service account ===
    # Attacker cracks svc_backup password and uses it
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "WS-ENG-001",
        "dst_host": "FS01",
        "logon_type": 3,
        "time": "2024-01-15 11:00:00",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "NTLM",
        "attack_phase": "privilege_escalation",
    },  # Service account from workstation - ANOMALY!
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "WS-ENG-001",
        "dst_host": "BACKUP01",
        "logon_type": 3,
        "time": "2024-01-15 11:05:00",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "NTLM",
        "attack_phase": "privilege_escalation",
    },
    # === LATERAL MOVEMENT - PsExec ===
    # PsExec to file server
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "WS-ENG-001",
        "dst_host": "FS01",
        "logon_type": 3,
        "time": "2024-01-15 11:30:00",
        "src_ip": "10.2.4.10",
        "status": "success",
        "auth_package": "NTLM",
        "attack_phase": "lateral_movement",
    },
    {
        "event_id": 7045,
        "service_name": "PSEXESVC",
        "host": "FS01",
        "user": "svc_backup",
        "time": "2024-01-15 11:30:05",
        "attack_phase": "lateral_movement",
    },
    # Server-to-server lateral movement (highly suspicious)
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "FS01",
        "dst_host": "FS02",
        "logon_type": 3,
        "time": "2024-01-15 11:35:00",
        "src_ip": "10.1.2.10",
        "status": "success",
        "auth_package": "NTLM",
        "attack_phase": "lateral_movement",
    },
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "FS01",
        "dst_host": "BACKUP01",
        "logon_type": 3,
        "time": "2024-01-15 11:40:00",
        "src_ip": "10.1.2.10",
        "status": "success",
        "auth_package": "NTLM",
        "attack_phase": "lateral_movement",
    },
    # === CREDENTIAL DUMPING on backup server ===
    # LSASS access for credential theft
    {
        "event_id": 4624,
        "user": "svc_backup",
        "src_host": "BACKUP01",
        "dst_host": "BACKUP01",
        "logon_type": 2,
        "time": "2024-01-15 12:00:00",
        "src_ip": "10.1.6.10",
        "status": "success",
        "auth_package": "Negotiate",
        "attack_phase": "credential_access",
    },
    {
        "event_id": 4656,
        "user": "svc_backup",
        "object": "lsass.exe",
        "access": "0x1010",
        "host": "BACKUP01",
        "time": "2024-01-15 12:05:00",
        "attack_phase": "credential_access",
    },
    # === DOMAIN ADMIN COMPROMISE ===
    # Attacker obtains da_john credentials from memory
    {
        "event_id": 4624,
        "user": "da_john",
        "src_host": "BACKUP01",
        "dst_host": "DC01",
        "logon_type": 3,
        "time": "2024-01-15 12:30:00",
        "src_ip": "10.1.6.10",
        "status": "success",
        "auth_package": "Kerberos",
        "attack_phase": "domain_compromise",
    },  # DA from backup server - CRITICAL!
    {
        "event_id": 4624,
        "user": "da_john",
        "src_host": "BACKUP01",
        "dst_host": "DC02",
        "logon_type": 3,
        "time": "2024-01-15 12:30:30",
        "src_ip": "10.1.6.10",
        "status": "success",
        "auth_package": "Kerberos",
        "attack_phase": "domain_compromise",
    },
    # === WMI LATERAL MOVEMENT ===
    {
        "event_id": 4624,
        "user": "da_john",
        "src_host": "DC01",
        "dst_host": "SCCM01",
        "logon_type": 3,
        "time": "2024-01-15 12:45:00",
        "src_ip": "10.1.1.10",
        "status": "success",
        "auth_package": "Kerberos",
        "attack_phase": "lateral_movement",
    },
    {
        "event_id": 5857,
        "host": "SCCM01",
        "provider": "WMI",
        "operation": "ExecMethod",
        "time": "2024-01-15 12:45:05",
        "attack_phase": "lateral_movement",
    },
    # === RDP LATERAL MOVEMENT ===
    {
        "event_id": 4624,
        "user": "da_john",
        "src_host": "SCCM01",
        "dst_host": "EXCH01",
        "logon_type": 10,
        "time": "2024-01-15 13:00:00",
        "src_ip": "10.1.7.10",
        "status": "success",
        "auth_package": "Negotiate",
        "attack_phase": "lateral_movement",
    },
    {
        "event_id": 4778,
        "user": "da_john",
        "src_host": "SCCM01",
        "dst_host": "EXCH01",
        "time": "2024-01-15 13:00:05",
        "attack_phase": "lateral_movement",
    },
    # === DCSYNC ATTACK ===
    {
        "event_id": 4662,
        "user": "da_john",
        "object_type": "domainDNS",
        "operation": "DS-Replication-Get-Changes-All",
        "host": "DC01",
        "time": "2024-01-15 13:30:00",
        "src_ip": "10.1.7.10",
        "attack_phase": "dcsync",
    },
    # === GOLDEN TICKET USAGE ===
    # Impossible travel - forged ticket from unknown location
    {
        "event_id": 4624,
        "user": "krbtgt",
        "src_host": "UNKNOWN",
        "dst_host": "DC01",
        "logon_type": 3,
        "time": "2024-01-15 14:00:00",
        "src_ip": "192.168.100.50",
        "status": "success",
        "auth_package": "Kerberos",
        "attack_phase": "golden_ticket",
    },
    {
        "event_id": 4769,
        "user": "Administrator",
        "src_host": "UNKNOWN",
        "service": "cifs/FS01",
        "time": "2024-01-15 14:00:30",
        "src_ip": "192.168.100.50",
        "encryption": "0x12",
        "ticket_options": "0x40810010",
        "attack_phase": "golden_ticket",
    },
    # === FAILED ATTEMPTS (Brute Force) ===
    {
        "event_id": 4625,
        "user": "administrator",
        "src_host": "EXTERNAL",
        "dst_host": "VPN01",
        "logon_type": 3,
        "time": "2024-01-15 03:00:00",
        "src_ip": "185.143.223.100",
        "status": "failure",
        "failure_reason": "Bad Password",
    },
    {
        "event_id": 4625,
        "user": "administrator",
        "src_host": "EXTERNAL",
        "dst_host": "VPN01",
        "logon_type": 3,
        "time": "2024-01-15 03:00:01",
        "src_ip": "185.143.223.100",
        "status": "failure",
        "failure_reason": "Bad Password",
    },
    {
        "event_id": 4625,
        "user": "administrator",
        "src_host": "EXTERNAL",
        "dst_host": "VPN01",
        "logon_type": 3,
        "time": "2024-01-15 03:00:02",
        "src_ip": "185.143.223.100",
        "status": "failure",
        "failure_reason": "Bad Password",
    },
    {
        "event_id": 4625,
        "user": "admin",
        "src_host": "EXTERNAL",
        "dst_host": "VPN01",
        "logon_type": 3,
        "time": "2024-01-15 03:00:03",
        "src_ip": "185.143.223.100",
        "status": "failure",
        "failure_reason": "No Such User",
    },
]

# Remote execution events
REMOTE_EXEC_EVENTS = [
    # PsExec indicators
    {
        "event_id": 7045,
        "service_name": "PSEXESVC",
        "host": "FS01",
        "user": "svc_backup",
        "time": "2024-01-15 11:30:05",
        "technique": "T1569.002",
    },
    {
        "event_id": 5145,
        "share": "\\\\FS01\\ADMIN$",
        "user": "svc_backup",
        "access": "WriteData",
        "time": "2024-01-15 11:30:03",
        "src_ip": "10.2.4.10",
        "technique": "T1021.002",
    },
    # WMI execution
    {
        "event_id": 5857,
        "host": "SCCM01",
        "provider": "WMI",
        "operation": "ExecMethod",
        "user": "da_john",
        "time": "2024-01-15 12:45:05",
        "technique": "T1047",
    },
    {
        "event_id": 5861,
        "host": "SCCM01",
        "query": "SELECT * FROM Win32_Process",
        "user": "da_john",
        "time": "2024-01-15 12:45:10",
        "technique": "T1047",
    },
    {
        "event_id": 4688,
        "process": "wmiprvse.exe",
        "cmdline": "wmic /node:DB01 process call create",
        "host": "SCCM01",
        "user": "da_john",
        "time": "2024-01-15 12:46:00",
        "technique": "T1047",
    },
    # WinRM execution
    {
        "event_id": 4688,
        "process": "wsmprovhost.exe",
        "host": "EXCH01",
        "user": "da_john",
        "time": "2024-01-15 13:05:00",
        "technique": "T1021.006",
    },
    {
        "event_id": 91,
        "host": "EXCH01",
        "message": "WinRM session created",
        "user": "da_john",
        "time": "2024-01-15 13:05:05",
        "technique": "T1021.006",
    },
    # SMB/Admin share access
    {
        "event_id": 5140,
        "share": "\\\\DC01\\C$",
        "user": "da_john",
        "access": "ReadWrite",
        "time": "2024-01-15 12:32:00",
        "src_ip": "10.1.6.10",
        "technique": "T1021.002",
    },
    {
        "event_id": 5145,
        "share": "\\\\DC01\\SYSVOL",
        "user": "da_john",
        "access": "WriteData",
        "time": "2024-01-15 12:35:00",
        "src_ip": "10.1.6.10",
        "technique": "T1021.002",
    },
    # Scheduled Task remote creation
    {
        "event_id": 4698,
        "task_name": "\\Microsoft\\Windows\\Maintenance\\WinSAT",
        "host": "DB01",
        "user": "da_john",
        "time": "2024-01-15 13:15:00",
        "technique": "T1053.005",
    },
    {
        "event_id": 4688,
        "process": "schtasks.exe",
        "cmdline": "schtasks /create /s DB01 /tn UpdateTask /tr C:\\temp\\payload.exe",
        "host": "SCCM01",
        "user": "da_john",
        "time": "2024-01-15 13:14:55",
        "technique": "T1053.005",
    },
    # Remote service creation
    {
        "event_id": 7045,
        "service_name": "WindowsUpdate",
        "host": "DB01",
        "user": "da_john",
        "image_path": "C:\\Windows\\Temp\\update.exe",
        "time": "2024-01-15 13:20:00",
        "technique": "T1543.003",
    },
    # DCOM execution
    {
        "event_id": 4688,
        "process": "mmc.exe",
        "cmdline": "mmc.exe -Embedding",
        "host": "WEB01",
        "parent": "svchost.exe",
        "time": "2024-01-15 13:25:00",
        "technique": "T1021.003",
    },
]

df = pd.DataFrame(AUTH_EVENTS)
df["time"] = pd.to_datetime(df["time"])

print("=" * 70)
print("ENTERPRISE LATERAL MOVEMENT ANALYSIS")
print("=" * 70)

print(f"\n🏢 Enterprise Topology: {len(ENTERPRISE_HOSTS)} hosts")
print(f"👤 User Accounts: {len(USERS)} accounts")
print(f"📋 Authentication Events: {len(AUTH_EVENTS)} events")
print(f"🔧 Remote Execution Events: {len(REMOTE_EXEC_EVENTS)} events")

# Count by attack phase
attack_events = [e for e in AUTH_EVENTS if "attack_phase" in e]
phases = {}
for e in attack_events:
    phase = e["attack_phase"]
    phases[phase] = phases.get(phase, 0) + 1

print("\n🔴 ATTACK CHAIN PHASES DETECTED:")
for phase, count in phases.items():
    print(f"   {phase}: {count} events")

## 2. Detecting Anomalous Authentication

In [ ]:
class LateralMovementDetector:
    """Comprehensive lateral movement and authentication anomaly detection."""

    def __init__(self, hosts: Dict, users: Dict):
        self.hosts = hosts
        self.users = users
        self.anomalies = []
        self.baseline = defaultdict(lambda: defaultdict(set))  # user -> host -> times

    def analyze_events(self, events: List[Dict]) -> List[Dict]:
        """Run all detection modules on authentication events."""
        self._detect_service_account_misuse(events)
        self._detect_server_lateral_movement(events)
        self._detect_kerberoasting(events)
        self._detect_domain_admin_anomalies(events)
        self._detect_golden_ticket(events)
        self._detect_dcsync(events)
        self._detect_brute_force(events)
        self._detect_unusual_auth_patterns(events)
        self._detect_admin_share_abuse(events)
        return self.anomalies

    def _detect_service_account_misuse(self, events: List[Dict]):
        """Service accounts authenticating from unexpected sources."""
        service_accounts = {u for u, d in self.users.items() if d["type"] == "service"}
        workstations = {h for h, d in self.hosts.items() if d["type"] == "workstation"}

        for event in events:
            if event.get("event_id") == 4624:
                user = event.get("user", "")
                src = event.get("src_host", "")

                if user in service_accounts and src in workstations:
                    self.anomalies.append(
                        {
                            "type": "service_account_misuse",
                            "severity": "CRITICAL",
                            "technique": "T1078.002",
                            "description": f"Service account '{user}' authenticated from workstation '{src}'",
                            "user": user,
                            "src_host": src,
                            "dst_host": event.get("dst_host"),
                            "time": event.get("time"),
                            "detection": "Service accounts should only auth from designated servers",
                        }
                    )

    def _detect_server_lateral_movement(self, events: List[Dict]):
        """Server-to-server lateral movement (unusual pattern)."""
        servers = {h for h, d in self.hosts.items() if d["type"] not in ["workstation"]}

        for event in events:
            if event.get("event_id") == 4624:
                src = event.get("src_host", "")
                dst = event.get("dst_host", "")

                # Server initiating connection to another server (not DC)
                src_info = self.hosts.get(src, {})
                dst_info = self.hosts.get(dst, {})

                if (
                    src_info.get("type")
                    not in ["workstation", "domain_controller", "jump_server", None]
                    and dst_info.get("type") not in ["domain_controller", None]
                    and src != dst
                ):
                    self.anomalies.append(
                        {
                            "type": "server_lateral_movement",
                            "severity": "HIGH",
                            "technique": "T1021",
                            "description": f"Lateral movement from server '{src}' to '{dst}'",
                            "user": event.get("user"),
                            "src_host": src,
                            "dst_host": dst,
                            "time": event.get("time"),
                            "detection": "Servers typically don't initiate connections to other servers",
                        }
                    )

    def _detect_kerberoasting(self, events: List[Dict]):
        """Multiple TGS requests in short timeframe."""
        tgs_requests = [e for e in events if e.get("event_id") == 4769]

        # Group by user and time window
        user_requests = defaultdict(list)
        for req in tgs_requests:
            user_requests[req.get("user", "")].append(req)

        for user, requests in user_requests.items():
            if len(requests) >= 3:  # Multiple service ticket requests
                # Check for RC4 encryption (0x17) - preferred by attackers
                rc4_requests = [r for r in requests if r.get("encryption") == "0x17"]

                if len(rc4_requests) >= 2:
                    self.anomalies.append(
                        {
                            "type": "kerberoasting",
                            "severity": "CRITICAL",
                            "technique": "T1558.003",
                            "description": f"User '{user}' made {len(requests)} TGS requests with RC4 encryption",
                            "user": user,
                            "services": [r.get("service") for r in requests],
                            "time": requests[0].get("time"),
                            "detection": "Multiple RC4 TGS requests indicate Kerberoasting attack",
                        }
                    )

    def _detect_domain_admin_anomalies(self, events: List[Dict]):
        """Domain admin accounts used from unusual locations."""
        domain_admins = {u for u, d in self.users.items() if d.get("type") == "domain_admin"}
        jump_servers = {h for h, d in self.hosts.items() if d.get("type") == "jump_server"}

        for event in events:
            if event.get("event_id") == 4624:
                user = event.get("user", "")
                src = event.get("src_host", "")

                if user in domain_admins and src not in jump_servers:
                    src_info = self.hosts.get(src, {})
                    if src_info.get("type") not in ["domain_controller", "jump_server"]:
                        self.anomalies.append(
                            {
                                "type": "domain_admin_anomaly",
                                "severity": "CRITICAL",
                                "technique": "T1078.002",
                                "description": f"Domain Admin '{user}' used from non-jump server '{src}'",
                                "user": user,
                                "src_host": src,
                                "dst_host": event.get("dst_host"),
                                "time": event.get("time"),
                                "detection": "Domain Admins should only authenticate from designated jump servers",
                            }
                        )

    def _detect_golden_ticket(self, events: List[Dict]):
        """Golden ticket indicators."""
        for event in events:
            # krbtgt account usage
            if event.get("user") == "krbtgt":
                self.anomalies.append(
                    {
                        "type": "golden_ticket",
                        "severity": "CRITICAL",
                        "technique": "T1558.001",
                        "description": "Potential Golden Ticket - krbtgt account authentication detected",
                        "src_host": event.get("src_host"),
                        "src_ip": event.get("src_ip"),
                        "time": event.get("time"),
                        "detection": "krbtgt account should never be used for interactive authentication",
                    }
                )

            # Unknown source with ticket
            if event.get("src_host") == "UNKNOWN" and event.get("event_id") in [4624, 4769]:
                self.anomalies.append(
                    {
                        "type": "forged_ticket",
                        "severity": "CRITICAL",
                        "technique": "T1558.001",
                        "description": f"Authentication from unknown host with IP {event.get('src_ip')}",
                        "user": event.get("user"),
                        "src_ip": event.get("src_ip"),
                        "time": event.get("time"),
                        "detection": "Authentication from unregistered host may indicate forged tickets",
                    }
                )

    def _detect_dcsync(self, events: List[Dict]):
        """DCSync attack detection."""
        for event in events:
            if event.get("event_id") == 4662:
                operation = event.get("operation", "")
                if "Replication" in operation and "Changes" in operation:
                    self.anomalies.append(
                        {
                            "type": "dcsync",
                            "severity": "CRITICAL",
                            "technique": "T1003.006",
                            "description": f"DCSync replication request from {event.get('src_ip')}",
                            "user": event.get("user"),
                            "host": event.get("host"),
                            "operation": operation,
                            "time": event.get("time"),
                            "detection": "Replication rights used from non-DC - potential DCSync attack",
                        }
                    )

    def _detect_brute_force(self, events: List[Dict]):
        """Brute force attack detection."""
        failures = [e for e in events if e.get("event_id") == 4625]

        # Group by source IP
        by_ip = defaultdict(list)
        for f in failures:
            by_ip[f.get("src_ip", "")].append(f)

        for ip, attempts in by_ip.items():
            if len(attempts) >= 3:
                users_tried = set(a.get("user") for a in attempts)
                self.anomalies.append(
                    {
                        "type": "brute_force",
                        "severity": "HIGH",
                        "technique": "T1110",
                        "description": f"Brute force from {ip}: {len(attempts)} failures for {len(users_tried)} users",
                        "src_ip": ip,
                        "users_attempted": list(users_tried),
                        "attempt_count": len(attempts),
                        "time": attempts[0].get("time"),
                        "detection": "Multiple authentication failures from same source",
                    }
                )

    def _detect_unusual_auth_patterns(self, events: List[Dict]):
        """Unusual authentication patterns (NTLM when Kerberos expected)."""
        for event in events:
            if event.get("event_id") == 4624:
                auth = event.get("auth_package", "")
                dst = event.get("dst_host", "")

                dst_info = self.hosts.get(dst, {})

                # NTLM to domain controller is suspicious
                if auth == "NTLM" and dst_info.get("type") == "domain_controller":
                    self.anomalies.append(
                        {
                            "type": "ntlm_to_dc",
                            "severity": "MEDIUM",
                            "technique": "T1550.002",
                            "description": f"NTLM authentication to domain controller {dst}",
                            "user": event.get("user"),
                            "src_host": event.get("src_host"),
                            "dst_host": dst,
                            "time": event.get("time"),
                            "detection": "NTLM to DC may indicate pass-the-hash or relay attack",
                        }
                    )

    def _detect_admin_share_abuse(self, events: List[Dict]):
        """Admin share (C$, ADMIN$) access from unusual sources."""
        for event in events:
            if event.get("event_id") in [5140, 5145]:
                share = event.get("share", "")
                if "C$" in share or "ADMIN$" in share:
                    self.anomalies.append(
                        {
                            "type": "admin_share_access",
                            "severity": "HIGH",
                            "technique": "T1021.002",
                            "description": f"Admin share access: {share}",
                            "user": event.get("user"),
                            "share": share,
                            "access": event.get("access"),
                            "src_ip": event.get("src_ip"),
                            "time": event.get("time"),
                            "detection": "Admin share access may indicate lateral movement",
                        }
                    )


# Run comprehensive detection
detector = LateralMovementDetector(ENTERPRISE_HOSTS, USERS)
all_events = AUTH_EVENTS + REMOTE_EXEC_EVENTS
anomalies = detector.analyze_events(all_events)

print("\n" + "=" * 70)
print("LATERAL MOVEMENT DETECTION RESULTS")
print("=" * 70)

# Group by severity
by_severity = defaultdict(list)
for a in anomalies:
    by_severity[a["severity"]].append(a)

print(f"\n📊 Total Anomalies: {len(anomalies)}")
for sev in ["CRITICAL", "HIGH", "MEDIUM", "LOW"]:
    if sev in by_severity:
        print(f"   {sev}: {len(by_severity[sev])}")

# Group by type
by_type = defaultdict(list)
for a in anomalies:
    by_type[a["type"]].append(a)

print(f"\n📋 Anomalies by Type:")
for atype, items in sorted(by_type.items(), key=lambda x: -len(x[1])):
    print(f"   {atype}: {len(items)}")

print("\n🔴 CRITICAL FINDINGS:")
for a in by_severity.get("CRITICAL", [])[:8]:
    print(f"\n   [{a['technique']}] {a['type'].upper()}")
    print(f"   {a['description']}")
    print(f"   Time: {a.get('time')}")
    print(f"   Detection: {a.get('detection')}")

## 3. Graph-Based Attack Path Analysis

In [ ]:
def build_auth_graph(events: List[Dict]) -> nx.DiGraph:
    """Build authentication graph from events."""
    G = nx.DiGraph()

    for event in events:
        if event["event_id"] == 4624:  # Successful login
            src = event["src_host"]
            dst = event["dst_host"]

            # Add edge with user as attribute
            if G.has_edge(src, dst):
                G[src][dst]["users"].add(event["user"])
                G[src][dst]["count"] += 1
            else:
                G.add_edge(src, dst, users={event["user"]}, count=1)

    return G


# Build graph
G = build_auth_graph(AUTH_EVENTS)

# Visualize
plt.figure(figsize=(10, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=2000, font_size=10, arrows=True)
plt.title("Authentication Flow Graph")
plt.show()

# Find potential attack paths
print("\nPotential attack paths from WS001:")
for target in ["DC01", "DB01"]:
    try:
        paths = list(nx.all_simple_paths(G, "WS001", target))
        for path in paths:
            print(f"  {' -> '.join(path)}")
    except nx.NetworkXNoPath:
        print(f"  No path to {target}")

## 4. Remote Execution Detection

In [ ]:
# Windows events indicating remote execution
REMOTE_EXEC_EVENTS = [
    {"event_id": 7045, "service_name": "PSEXESVC", "host": "DC01", "user": "admin"},  # PsExec
    {
        "event_id": 4688,
        "process": "wmiprvse.exe",
        "cmdline": "wmic /node:FS01",
        "host": "WS001",
    },  # WMI
    {"event_id": 4688, "process": "wsmprovhost.exe", "host": "DB01"},  # WinRM
]

REMOTE_EXEC_INDICATORS = {
    "PSEXESVC": {"technique": "T1569.002", "tool": "PsExec"},
    "wmiprvse.exe": {"technique": "T1047", "tool": "WMI"},
    "wsmprovhost.exe": {"technique": "T1021.006", "tool": "WinRM"},
}


def detect_remote_execution(events: List[Dict]) -> List[Dict]:
    """Detect remote execution techniques."""
    detections = []

    for event in events:
        for indicator, info in REMOTE_EXEC_INDICATORS.items():
            if indicator in str(event.values()):
                detections.append(
                    {
                        "host": event.get("host"),
                        "tool": info["tool"],
                        "technique": info["technique"],
                        "event": event,
                    }
                )

    return detections


remote_exec = detect_remote_execution(REMOTE_EXEC_EVENTS)
print("Remote Execution Detections:")
for r in remote_exec:
    print(f"  {r['host']}: {r['tool']} ({r['technique']})")

## 5. First-Time Access Detection

In [ ]:
class FirstTimeAccessDetector:
    """Detect first-time user access to hosts."""

    def __init__(self):
        self.baseline = defaultdict(set)  # user -> set of hosts

    def train(self, events: List[Dict]):
        """Build baseline from historical data."""
        for event in events:
            if event["event_id"] == 4624:
                self.baseline[event["user"]].add(event["dst_host"])

    def detect(self, event: Dict) -> bool:
        """Check if this is first-time access."""
        if event["event_id"] != 4624:
            return False
        return event["dst_host"] not in self.baseline.get(event["user"], set())


# Demo
detector = FirstTimeAccessDetector()
detector.train(AUTH_EVENTS[:3])  # Train on first 3 events

# Check remaining events
print("First-time access checks:")
for event in AUTH_EVENTS[3:]:
    if event["event_id"] == 4624:
        is_first = detector.detect(event)
        print(f"  {event['user']} -> {event['dst_host']}: {'FIRST TIME!' if is_first else 'Known'}")

## Key Takeaways

1. **Auth Anomalies**: Service accounts, server-to-server movement
2. **Graph Analysis**: Visualize and analyze attack paths
3. **Remote Execution**: PsExec, WMI, WinRM indicators
4. **First-Time Access**: Baseline-based anomaly detection

## Next Steps
- **Lab 16**: Threat Actor Profiling
- **Lab 17**: Adversarial ML